In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import transformers
from transformers import pipeline
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import numpy as np
import nltk.data
from requests_html import HTMLSession

In [53]:
def parse_href(url, element):
    session = HTMLSession()
    r = session.get(url)
    elements = r.html.find(element)
    links = [element.attrs['href'] for element in elements if 'href' in element.attrs]
    return links


In [ ]:
hyperlinks = parse_href(url, 'a')

# Print the string versions of the hyperlinks
for hyperlink in hyperlinks:
    print(hyperlink)


In [55]:
def parse_elements(url,element):
    session = HTMLSession()
    r = session.get(url)
    elements = r.html.find(element)
    return elements

In [56]:
paragraphs = parse_elements(url, 'p')
links = parse_elements(url, 'a')

In [57]:
def topic_detection(sentence):
    DataScience_words = ('data science', 'regression', 'predict', 'predictive', 'classify')
    AI_words = ('AI', 'machine learning', 'ml', 'ML', 'NLP', 'LLM','deep learning','neural network','generative', 'GPT-3')
    DataWords = ('analysis', 'algorithms','data','data structure','data-structure','big data')
    DataScience = any(sentence.count(i) > 0 for i in DataScience_words)
    AI = any(sentence.count(i) > 0 for i in AI_words)
    data = any(sentence.count(i) > 0 for i in DataWords)
    topics = []
    if DataScience == True:
        topics.append("Data Science")
    if AI == True:
        topics.append("AI")
    if data == True:
        topics.append("data")
    return topics

In [58]:
p_dictionary = {}
p_list = []
topic_list = []
for p in range(0, len(paragraphs)):
    if len(paragraphs[p].text) > 50:
      #print((paragraphs[p].text))
      #print(f"TALKS ABOUT: {topic_detection(paragraphs[p].text)}")
      p_list.append(paragraphs[p].text)
      topic_list.append(topic_detection(paragraphs[p].text))
p_dictionary['Paragraphs'] = p_list
p_dictionary['Topics'] = topic_list

In [59]:
p_dictionary

{'Paragraphs': ['Home Depot beats earnings estimates but warns of "continued pressure" on consumers'],
 'Topics': [[]]}

In [60]:
medium_df = pd.DataFrame(p_dictionary)

In [ ]:
medium_df

In [18]:
medium_df['Character Count'] = medium_df['Paragraphs'].apply(lambda x: len(x))


# Start

In [2]:
model_id = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
label = pipeline('sentiment-analysis', model = model_id)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [3]:
HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }


In [76]:
url = 'https://financialpost.com/news/economy/canada-inflation-rate-higher-than-expected'
response = requests.get(url, headers=HEADERS)
page_content = response.text
doc = BeautifulSoup(page_content, 'html.parser')
header = []
news_elements = doc.find_all('h3', {"class" : 'article-card__headline m-0 article-card__headline--clamp-3'})
news_header = [] 
for i in range (len(news_elements)):
    news_header.append(news_elements[i].text)

In [78]:
len(news_header)


5

In [6]:
for i in range (len(a_elements)):
    header.append(a_elements[i].text)
news_header_df = pd.DataFrame({'News_Header': header})

In [7]:
classification = []
for i in range (len(news_header_df.News_Header)):
    classification.append(label(news_header_df.News_Header[i]))

In [28]:
classification[4][0]['label']

'neutral'

In [32]:
labels = []
for i in range (len(classification)):
    labels.append(classification[i][0]['label'])

In [33]:
news_header_df['sentiment_analysis'] = labels